# Import 3-Coin Returns

In [12]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3) 
from scipy.optimize import minimize     #Optimization library
from numpy import genfromtxt


In [13]:
nAssets = 3                     # Number of assets
my_data = genfromtxt('returns_coins.csv', delimiter=',')

my_ret = np.array(my_data)
Return = my_ret[1:,1:4]
Return.shape

(15120, 3)

# MVO optimization model

In [14]:
def objective(wgt):                         # needed for minimize function below
    tol = 15                                # risk tolerance can be changed from conservative to aggressive
    obj = -(np.dot(wgt, ret) - (np.dot(wgt, np.dot(covMat, wgt)))/(2*tol))     # Objective function
    return obj

def constraint1(wgt):                        # needed for minimize function below
    cnst1 = wgt[0] + wgt[1] + wgt[2] - 1
    return cnst1


# Use MVO model to generate weights on each rebalance period

In [15]:
# define length of rebalence period
rebalance_days = 21  

In [16]:
b = (0,None)                                 # bounds: weight >= 0
bnds = (b,b,b)
con1 = {'type': 'eq', 'fun': constraint1}    # needed for minimize function below

nObs = Return.shape[0]
period = round(nObs/rebalance_days)
wgt = np.array([1, 0, 0])           # initial weight
memo = []

for i in range (period):   
    
    train_i = Return[i*rebalance_days : i*rebalance_days+rebalance_days]
    ret = np.mean(train_i,axis=0)                                   # Expected returns (stocks, bonds, and cash)
    sdMat = np.diag(np.std(train_i,axis=0))                         # Standard deviations     
    #corrMat = np.corrcoef(meanRet.transpose())
    corrMat = np.reshape(np.corrcoef(train_i.transpose()), (3,3))   # Correlations
    covMat = np.dot(sdMat, np.dot(corrMat, sdMat))                  # Covariance matrix (sd * corr * sd)

    
    # check if the covariance matrix is PSD
    ev = np.linalg.eigvals(corrMat)                   # Eigenvalues of correlation matrix
    if sum(n < 0 for n in ev):                        # If eigenvalues are negative then corrMat NOT PSD
        # print('*** Error: Correlation matrix is NOT Positive Semi-Definite (PSD)  ***')
        print('\033[1;31m*** Error: Correlation matrix is NOT Positive Semi-Definite (PSD)  ***\033[1;m]')
        print()
        print('Eigenvalues =', ev)
        print()

    
    # Run Optimizer
    solution = minimize(objective, wgt, method='SLSQP', bounds=bnds, constraints=con1)
    
    # Record each month's allocation strategy
    memo.append(solution.x)


In [17]:
# check weight
memo_wgt = np.reshape(np.round(memo,2), (period,3))
print(memo_wgt[2:10])
len(memo_wgt)

[[1.  0.  0. ]
 [0.1 0.  0.9]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [1.  0.  0. ]
 [0.  0.  1. ]]


720

In [18]:
# export MVO generated weights
df = pd.DataFrame(memo_wgt)
df.to_csv('coins_weight_train.csv', index=None, header=True)
df.to_csv('coins_weight_test.csv', index=None, header=True)